##**Gerrymandering-Environment**

    INITIAL STATE (provided externally via reset(options=...)):
        - 'district_map'
        - 'social_graph'
        - 'opinions'     

    ACTION:
        - new district assignment for each voter

    OBSERVATION (returned by reset/step):
        {
          'district_map'   : (num_voters,)
          'representatives': (num_districts,)  # voter indices; -1 if empty   district
          'social_graph'   : (num_voters, num_voters)  # AUGMENTED: base social + rep->voter edges used for the step
          'opinions'       : (num_voters, 2)
          'opinion_graph'  : (num_voters, num_voters)  # similarity kernel derived from opinion distances
        }

    KEY LOGIC:
      - Representatives: for each district, pick the member that minimizes the sum of L2 distances to members in that district (discrete 1-median).
      - Opinion dynamics: DRF (assimilation/neutral/backfire) with weighted neighbor influence.
      - Reward: reduction in total distance to reference opinion c*


In [1]:
!pip install torch_geometric

In [2]:
import numpy as np
import torch
from torch_geometric.data import Data
import gymnasium as gym
from gymnasium import spaces
import copy
from typing import Optional, Tuple, Dict, Any
import math

#This is the Frankenmandering Data class which includes:
*   social edge list: type: matrix and size : (v,v)
*   assignment edge list: type: matrix and size:(v,d)
*   number of orignin edge
*   geometric position
*   district label
*   edge attribute
*   representative
*   opinion



In [3]:
class FrankenData(Data):
  def __init__(self, social_edge, geographical_edge, orig_edge_num, opinion, pos, reps, dist_label, edge_attr, geo_edge_attr, **kwargs):
    super().__init__(**kwargs)

    self.orig_edge_num = int(orig_edge_num)

    # core tensors
    self.opinion = torch.as_tensor(opinion, dtype=torch.float32)        # (N,1) or (N,m)
    self.pos = torch.as_tensor(pos, dtype=torch.float32)                # (N,2)
    self.dist_label = torch.as_tensor(dist_label, dtype=torch.long)     # (N,) **long**

    self.social_edge = torch.as_tensor(social_edge, dtype=torch.long)   # (2,E)
    self.edge_attr   = torch.as_tensor(edge_attr, dtype=torch.float32)  # (E,)

    # geographical edge is optional (allow None)
    self.geographical_edge = torch.as_tensor(geographical_edge, dtype=torch.long)
    self.geo_edge_attr   = torch.as_tensor(geo_edge_attr, dtype=torch.float32)  # (E,)

    self.reps = list(reps)  # list of length K-number_of_districts, each is int or None

    # kwargs may include things like: hetero=HeteroData
    for k, v in kwargs.items():
        setattr(self, k, v)

  @staticmethod
  def save(self,file_path):
      torch.save(self,file_path)
      print(f"FrankenData object saved to {file_path}")

  @staticmethod
  def load(self,file_path):
      data = torch.load(file_path)
      print(f"FrankenData object loaded from {file_path}")
      return data

In [5]:
class FrankenmanderingEnv(gym.Env):

    metadata = {"render_modes": ["human"]}

    def __init__(
        self,
        num_voters: int,
        num_districts: int,
        opinion_dim: int = 2,
        horizon: int = 10,
        seed: int | None = None,
        FrankenData = None,
        # target
        target_opinion: np.ndarray | None = None,

    ):
        super().__init__()
        self.num_voters = int(num_voters)
        self.num_districts = int(num_districts)
        self.opinion_dim = int(opinion_dim)
        self.horizon = int(horizon)
        self.rng = np.random.default_rng(seed)

        # state
        self.t = 0
        self.initial_data = copy.deepcopy(FrankenData)
        self.FrankenData = FrankenData
        self.opinion_update_step = 0

        # target opinion c*
        if target_opinion is None:
            self.c_star = np.zeros((self.num_voters,self.opinion_dim), dtype=np.float32)
        else:
            self.c_star = np.asarray(target_opinion, dtype=np.float32).reshape(self.num_voters,self.opinion_dim)

        # action spaces
        self.action_space = spaces.Box(low=0.0, high=1.0, shape=(self.num_voters, self.num_districts), dtype=np.float32)

        self.observation_space = spaces.Dict({
                  "opinions": spaces.Box(low=-np.inf, high=np.inf,
                                        shape=(self.num_voters, self.opinion_dim), dtype=np.float32),

                  "positions": spaces.Box(low=-np.inf, high=np.inf,
                                          shape=(self.num_voters, 2), dtype=np.float32),

                  "dist_label": spaces.Box(low=0, high=self.num_districts-1,
                                          shape=(self.num_voters,), dtype=np.int32),

                  "reps": spaces.Box(low=-1, high=self.num_voters-1,
                                    shape=(self.num_districts,), dtype=np.int32),

                  "social_edge_index": spaces.Box(low=0, high=self.num_voters-1,
                                shape=(2, self.initial_data.orig_edge_num), dtype=np.int32),

                  "social_edge_attr": spaces.Box(low=0, high=np.inf,
                                                shape=(self.initial_data.orig_edge_num,), dtype=np.float32),

                  "assignment": spaces.Box(low=0, high=self.num_voters-1,
                                          shape=(self.num_voters, self.num_districts), dtype=np.int32),
              })


    # options includes opinions, geometric position,  edge_index(social graph,assignment),assignment
    def reset(self, seed: int | None = None, options: dict | None = None):
        super().reset(seed=seed)
        self.t = 0

        self.FrankenData = copy.deepcopy(self.initial_data)

        return self.FrankenData, {}

    def step(self, action: np.ndarray, eps_indiff, eps_assim, eps_backfire , eps_irrel, eps_amb, assim_shift, back_shift, indiff_shift, amb_shift, irr_shift):
        if self.FrankenData is None:
            raise RuntimeError("Environment must be reset before calling step.")

        # Normalize action and compute new assignment
        assignment = row_normalize(np.asarray(action, dtype=np.float32))

        # assignment = ensure_non_empty(assignment)
        dist_label = assignment.argmax(axis=1).astype(np.int32)
        dist_label[(assignment.max(axis=1) == 0)] = -1

        # check the validity of district
        # contiguous = check_contiguity(self.FrankenData.geographical_edge, y, self.num_districts)
        non_empty = all(np.any(dist_label == d) for d in range(self.num_districts))


        if  (not non_empty):
          terminated = False
          Invalid_dist_reward = -10
          return self.FrankenData, Invalid_dist_reward, terminated,False,{}

        # Elect new representatives
        reps = elect_representatives(dist_label, self.FrankenData.opinion, self.num_districts)
        #partial reset augmented graph

        social_graph = self.FrankenData.social_edge[:, :-self.FrankenData.orig_edge_num]

        social_edge_attr = self.FrankenData.edge_attr[:-self.FrankenData.orig_edge_num]

        # update social graph based on reps and new assignment
        aug_edge_index, aug_edge_attr = augment_with_reps(
            social_graph,
            social_edge_attr,
            reps,
            dist_label
        )

        # Update oppinion
        x_new = opinion_update(aug_edge_index, aug_edge_attr,self.FrankenData.opinion, eps_indiff, eps_assim, eps_backfire ,
        eps_irrel, eps_amb, assim_shift, back_shift, indiff_shift, amb_shift, irr_shift )

        reward = self.reward(self.FrankenData.opinion, x_new, dist_label, self.FrankenData.social_edge)

        self.t += 1
        self.opinion_update_step +=1
        terminated = self.t >= self.horizon

        self.FrankenData = FrankenData(
              social_edge = aug_edge_index,
              orig_edge_num = aug_edge_index.shape[1],
              geographical_edge= self.initial_data.geographical_edge,
              opinion= x_new,
              pos =self.FrankenData.pos,
              reps=[r if r is not None else -1 for r in reps],
              dist_label=dist_label,
              edge_attr = aug_edge_attr,
              geo_edge_attr= self.initial_data.geo_edge_attr
            )
        return self.FrankenData, float(reward), terminated, False, {}

    def reward(self, old_opinion,new_opinion, dist_label, assignment, a= 0.1, b =0.5):
        # check the population equity and compactness constraints
        pop_dev = population_equality(np.ones_like(dist_label),dist_label, self.num_districts)
        comp_score = compactness_score(assignment, dist_label)

        penalty =  -a * pop_dev - b * comp_score

        old_d=np.linalg.norm(old_opinion-self.c_star,axis=1).sum()
        new_d=np.linalg.norm(new_opinion-self.c_star,axis=1).sum()

        reward = penalty + old_d-new_d
        return reward

**Elect representatives**
###This function elects representative in each district using a spatial / metric voting rule, specifically the median voter rule

```
Args:
        y: (N,) array of district labels for each voter.
        X: (N, opinion_dim) array of voter opinions.
        num_districts: number of districts (K).

    Returns:
        reps: list of representative indices, one per district (None if district empty)
```



In [6]:
import numpy as np

def elect_representatives(dist_label: np.ndarray, opinion: np.ndarray, num_districts: int):

    reps = [None] * num_districts
    N, m = opinion.shape

    # Precompute pairwise distances
    dif = opinion[:, None, :] - opinion[None, :, :]   # (N,N,m)
    dists = np.linalg.norm(dif, axis=2)   # (N,N) Euclidean distances

    for d in range(num_districts):
        members = np.where(dist_label == d)[0]  # voters in district d

        if len(members) == 0:
            continue  # empty district

        # For each member, compute sum of distances to others in district
        sums = dists[np.ix_(members, members)].sum(axis=1)

        # Pick the member with the minimal total distance/it stores the index of opinion
        reps[d] = int(members[np.argmin(sums)])

    return reps

## **Augment with reps (G union H)**

###
Augment the social graph with representative edges (district graphs).
    
    Args:
        edge_index: (2, E) numpy array of existing social edges.
        edge_attr: (E,) numpy array of edge weights.
        reps: list of representative indices, one per district (None if empty).
        y: (N,) array of district assignments (each voter’s district label).
        rep_edge_weight: weight to assign to each representative edge (default=1.0).
    
    Returns:
        new_edge_index: (2, E+E_rep) with representative edges added.
        new_edge_attr:  (E+E_rep,) with weights for representative edges.

In [8]:

def augment_with_reps(edge_index, edge_attr, reps, dist_label, rep_edge_weight: float = 1.0):

    add_src, add_dst, add_w = [], [], []
    existing = set(map(tuple, edge_index.T))

    for d, r in enumerate(reps):
        if r is None:
            continue  # empty district
        members = np.where(dist_label == d)[0]

        for v in members:
            if v == r:
                continue  # skip self-loop

            if (r, v) in existing:
              continue  # skip duplicate

            add_src.append(r)             # representative -> member
            add_dst.append(v)
            add_w.append(rep_edge_weight) # fixed influence weight

    if not add_src:  # no additional edges
        return edge_index, edge_attr

    rep_edges = np.vstack([add_src, add_dst])      # shape (2, E_rep)
    rep_weights = np.array(add_w, dtype=np.float32)

    new_edge_index = np.concatenate([edge_index, rep_edges], axis=1)
    new_edge_attr  = np.concatenate([edge_attr, rep_weights], axis=0)

    return new_edge_index, new_edge_attr

### **Opinion Update Function**
### At each step, voters "adjust" their position in opinion space based on the Euclidean distance to their neighbours, with the DRF


deciding whether they converge, diverge, or ignore.

Each neighbour "pulls" or "pushes" v depending on how similar they are.

The closer they are, the more v tends to assimilate.

If they’re a bit too far, v resists and moves in the opposite direction (backfire).

If they’re too far away, v ignores them.

The update is multi-dimensional, so if opinions are about multiple issues, the shift happens along the direction where the neighbour differs most.

Because normalization is used, v doesn’t "overreact" to having many neighbours.

    Apply opinion update based on DRF across graph edges.
    X: (N, opinion_dim)
    edge_index (augmented social graph): shape (2, E)
    edge_attr(w): shape (E,)
    mu : μ(∥cut​−cvt​∥)

In [9]:
def opinion_update(edge_index_aug, edge_attr, opinion: np.ndarray,eps_indiff, eps_assim, eps_backfire , eps_irrel, eps_amb, assim_shift, back_shift, indiff_shift, amb_shift, irr_shift):

    N, m = opinion.shape   # N voters, m-dimensional opinions
    newX = np.copy(opinion)

    updates = np.zeros_like(opinion)
    norm_factors = np.zeros((N, 1)) #Z

    for e in range(edge_index_aug.shape[1]):
        u, v = edge_index_aug[:, e]
        w = edge_attr[e]

        diff = opinion[u] - opinion[v]               # vector difference in opinion space

       # Euclidean distance
       # if diff is a 1-D array [x, y, z], then np.linalg.norm(diff, 2) calculates sqrt(x^2 + y^2 + z^2)
        dist = np.linalg.norm(diff, 2)

        if dist > 1e-8:  # avoid division by zero
            # assim_shift = np.sign(dist)
            # back_shift = -np.sign(dist)
            mu = drf(dist,eps_indiff, eps_assim, eps_backfire , eps_irrel, eps_amb, assim_shift, back_shift, indiff_shift, amb_shift, irr_shift)

            direction = (diff / dist).numpy()      # unit vector (direction only)

            # influence: u influences v
            updates[v] += mu * w * direction
            norm_factors[v] += abs(w)

    # apply updates with normalization
    for v in range(N):
        if norm_factors[v] > 0:
            newX[v] = opinion[v] + updates[v] / norm_factors[v]

    return newX

**Discrepancy Response Function**
*   This function determines how a voter reacts when interacting with another voter based on the discrepancy (distance) between their opinions.
*   Tunable thresholds: epsilon values determine assimilation/backfire ranges. There are 5 levels in this range

### thresholds (tune these as hyperparameters)
*   eps_indiff = 0.5    below this = indifference
*   eps_backfire = 4.0  backfire zone
*   eps_irrel = 6.0     above this = irrelevance    
*   eps_assim = 2.0     assimilation zone

In [10]:
def drf(discrepancy, eps_indiff, eps_assim, eps_backfire , eps_irrel, eps_amb, assim_shift, back_shift, indiff_shift, amb_shift, irr_shift):
    delta = abs(discrepancy)

    if 0 <= delta <= eps_indiff:
        return indiff_shift  # indifference

    elif eps_indiff < delta < eps_assim:
        return assim_shift  # assimilation (pull closer)

    elif eps_backfire <= delta < eps_irrel:
        return back_shift  # backfire (push away)

    elif eps_irrel <= delta  :
        return irr_shift  # irrelevance (ignored)

    elif delta <= eps_amb:
        return amb_shift  # ambivalence

In [11]:
def row_normalize(A: np.ndarray, eps=1e-8) -> np.ndarray:
        # A = np.array(A)
        row_sum = A.sum(axis=1, keepdims=True)
        row_sum = np.clip(row_sum, eps, None)
        return A / row_sum


def adj_to_coo(adj: np.ndarray):
        row, col = np.nonzero(adj)
        edge_index = np.vstack([row, col])
        edge_attr = adj[row, col]
        return edge_index, edge_attr

## **Population equality**
###(districts must have nearly equal populations)
Compute population deviation score.
    
    population weight: np.ndarray shape (N,) -> population of each voter/node
    labels: np.ndarray shape (N,) -> district assignment
    num_districts: int
    
    Returns: float (sum of deviations from ideal)
    """
    Returns normalized population deviation:
      sum(|pop_d - ideal|) / total_pop
    This is in [0, 2*(1 - 1/num_districts)] roughly; smaller is better.
    """


In [12]:
def population_equality(pop_weights: np.ndarray, dist_label: np.ndarray, num_districts: int) -> float:

    pop_weights = np.asarray(pop_weights, dtype=float)
    total = pop_weights.sum()
    pop_per = np.zeros(num_districts, dtype=float)

    for d in range(num_districts):
        pop_per[d] = pop_weights[dist_label == d].sum()
    ideal = total / float(num_districts)

    return float(np.abs(pop_per - ideal).sum() / (total + 1e-12))

## **check contiguity**
###A district is contiguous if all its members are connected to each other in the graph.
This function Check whether each district forms a connected component.
    
    edge_index: np.ndarray shape (2, E) -> edges
    labels: np.ndarray shape (N,) -> district assignment of each node
    num_districts: int
    
    Returns: bool (True if all districts are contiguous)

## **compactness score**
###  Compactness means districts should not be “spread out” with too many edges crossing between districts.This function computes compactness as ratio of cut edges to total edges.
    
    edge_index (social connection): np.ndarray shape (2, E)
    labels (district labels): np.ndarray shape (N,)
    
    Returns: float (compactness score)

In [14]:
def compactness_score(geographical_edge, dist_label: np.ndarray) -> float:
    """
    Compactness = fraction of geographical edges that are cut (endpoints in different districts).
    Lower is more compact. Accepts same geographical_edge formats as build_neighbors().
    """
    ge = np.asarray(geographical_edge)
    # normalize edges to a list of pairs
    if ge.ndim == 2 and ge.shape[0] == ge.shape[1]:
        # adjacency matrix -> build unique edges (i<j)
        N = ge.shape[0]
        cut = 0
        total = 0
        for i in range(N):
            for j in range(i+1, N):
                if ge[i, j]:
                    total += 1
                    if dist_label[i] != dist_label[j]:
                        cut += 1
        return 0.0 if total == 0 else float(cut / total)
    else:
        if ge.shape[0] == 2:
            edges = ge.T.astype(int)
        else:
            edges = ge.astype(int)
        total = edges.shape[0]
        if total == 0:
            return 0.0
        cuts = np.sum([1 for (u, v) in edges if int(dist_label[int(u)]) != int(dist_label[int(v)])])
        return float(cuts / total)